In [90]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

%matplotlib inline

In [91]:
# Load .env environment variables
load_dotenv()

True

In [92]:
# Set alpaca api key and secret 
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)


In [93]:
# Set the tickers
microsoft = ["MSFT"]

In [94]:
# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

In [95]:
# Format current date as ISO format
today = pd.Timestamp("2022-06-01", tz="America/New_York").isoformat()

In [96]:
# Get current closing prices for MSFT
df_microsoft = alpaca.get_bars(
    microsoft,
    timeframe,
    start = today,
    end = today
).df

# Display sample data
df_microsoft

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2022-06-01 04:00:00+00:00,275,277.69,270.04,272.42,25108530,318649,273.328471,MSFT


In [97]:
# Format start and end dates as ISO format for five year period
start = pd.Timestamp("2017-06-01", tz="America/New_York").isoformat()
end = pd.Timestamp("2022-06-01", tz="America/New_York").isoformat()

In [100]:
# Get closing prices for MSFT and TSLA from the five years
df_microsoft_daily = alpaca.get_bars(
    microsoft,
    timeframe,
    start = start,
    end = end,
    limit = 10000
).df

# Drop unneccesary columns
df_microsoft_daily = df_microsoft_daily.drop(["trade_count", "vwap"], axis=1)
# Display sample data
df_microsoft_daily

,open,high,low,close,volume,symbol
timestamp,,,,,,
2017-06-01 04:00:00+00:00,70.24,70.61,69.4510,70.10,21922908,MSFT
2017-06-02 04:00:00+00:00,70.44,71.86,70.2400,71.76,35470261,MSFT
2017-06-05 04:00:00+00:00,71.97,72.89,71.8100,72.28,33317996,MSFT
2017-06-06 04:00:00+00:00,72.30,72.62,72.2700,72.52,31511065,MSFT
2017-06-07 04:00:00+00:00,72.67,72.77,71.9500,72.39,22302769,MSFT
...,...,...,...,...,...,...
2022-05-25 04:00:00+00:00,258.36,264.58,257.1250,262.52,28402776,MSFT
2022-05-26 04:00:00+00:00,262.29,267.11,261.4294,265.90,24933202,MSFT
2022-05-27 04:00:00+00:00,268.51,273.34,267.5600,273.24,26809462,MSFT


In [ ]:
# Reorganize the DataFrame
# Separate ticker data
#MSFT = df_portfolio_years[df_portfolio_years['symbol']=='MSFT'].drop('symbol', axis=1)
#TSLA = df_portfolio_years[df_portfolio_years['symbol']=='TSLA'].drop('symbol', axis=1)
# Concatenate the ticker DataFrames
#df_portfolio_years = pd.concat([MSFT, TSLA],axis=1, keys=['MSFT','TSLA'])
#df_portfolio_years

In [ ]:
# For loop to input any stock to get alpaca stock data

def get_company(ticker, start, end, tradeapi):
    return alpaca.get_bars(
        ticker,
        TimeFrame.Day,
        start,
        end
    ).df
